In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Libraries

In [163]:
#Importing all the librairies we'll need
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,r2_score
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV

In [164]:
train=pd.read_csv("/content/drive/MyDrive/ZINDI/IEEE Hackathon/Train_IEEE.csv")

In [165]:
test=pd.read_csv("/content/drive/MyDrive/ZINDI/IEEE Hackathon/Test_IEEE.csv")

In [166]:
train.dtypes

id                          object
asset_id                     int64
open                       float64
high                       float64
low                        float64
volume                     float64
market_cap                 float64
url_shares                 float64
unique_url_shares          float64
reddit_posts               float64
reddit_posts_score         float64
reddit_comments            float64
reddit_comments_score      float64
tweets                     float64
tweet_spam                 float64
tweet_followers            float64
tweet_quotes               float64
tweet_retweets             float64
tweet_replies              float64
tweet_favorites            float64
tweet_sentiment1           float64
tweet_sentiment2           float64
tweet_sentiment3           float64
tweet_sentiment4           float64
tweet_sentiment5           float64
tweet_sentiment_impact1    float64
tweet_sentiment_impact2    float64
tweet_sentiment_impact3    float64
tweet_sentiment_impa

In [167]:
corrs = train.corr()

In [168]:
corrs = corrs.sort_values('close', ascending = False)

# Ten most positive correlations
pd.DataFrame(corrs['close'].head(20))

,close
close,1.000000
market_cap,0.999978
high,0.999971
low,0.999956
open,0.999936
market_cap_global,0.998884
volume_24h_rank,0.787742
tweet_spam,0.751862
reddit_comments,0.712700
tweet_sentiment2,0.700992


In [169]:
# fill missing values
train = train.fillna(0)
test = test.fillna(0)

In [170]:
train

,id,asset_id,open,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_volume,percent_change_24h,market_cap_global,close
0,ID_322qz6,1,9422.849081,9428.490628,9422.849081,7.131986e+08,1.737635e+11,1689.0,817.0,55.0,105.0,61.0,271.0,3420.0,1671.0,11675867.0,39.0,1343.0,448.0,2237.0,124.0,330.0,331.0,2515.0,120.0,506133.0,1326610.0,1159677.0,8406185.0,281329.0,11681999.0,3.6,69.0,2.7,3.6,3.3,66.0,0.007118,1.0,606.0,2.0,1.0,1.0,2.0,5.0,4422,1.434516,2.818066e+11,9428.279323
1,ID_3239o9,1,7985.359278,7992.059917,7967.567267,4.004755e+08,1.426942e+11,920.0,544.0,20.0,531.0,103.0,533.0,1491.0,242.0,5917814.0,195.0,1070.0,671.0,3888.0,1.0,52.0,315.0,1100.0,23.0,1320.0,381117.0,1706376.0,3754815.0,80010.0,5924770.0,3.7,1.0,2.0,2.0,1.0,43.5,0.009419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2159,-2.459507,2.126897e+11,7967.567267
2,ID_323J9k,1,49202.033778,49394.593518,49068.057046,3.017729e+09,9.166977e+11,1446.0,975.0,72.0,1152.0,187.0,905.0,9346.0,4013.0,47778746.0,104.0,2014.0,1099.0,11476.0,331.0,923.0,864.0,6786.0,442.0,9848462.0,5178557.0,2145663.0,25510267.0,5110490.0,47796942.0,3.7,22.0,3.1,3.0,3.3,65.5,0.013530,1.0,692.0,3.0,1.0,1.0,0.0,0.0,10602,4.942448,1.530712e+12,49120.738484
3,ID_323y5P,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,17.0,424.0,268.0,443.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,285,0.000000,0.000000e+00,0.000000
4,ID_324kJH,1,10535.737119,10535.737119,10384.798216,1.150053e+09,1.921183e+11,1012.0,638.0,24.0,42.0,50.0,173.0,3262.0,1652.0,14422172.0,21.0,511.0,190.0,2284.0,86.0,280.0,443.0,2284.0,169.0,311017.0,1977833.0,731277.0,10964321.0,440730.0,14426405.0,3.7,22.0,4.7,3.8,4.4,83.0,0.010332,1.0,749.0,2.0,1.0,1.0,0.0,2.0,3996,2.609576,3.386925e+11,10384.798216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12627,ID_zybd4H,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,2.0,61.0,138.0,161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140,0.000000,0.000000e+00,0.000000
12628,ID_zydeys,1,7294.506136,7306.701928,7292.729876,3.816704e+08,1.322473e+11,605.0,462.0,16.0,217.0,39.0,156.0,1501.0,241.0,10909376.0,9.0,170.0,44.0,292.0,35.0,51.0,358.0,973.0,84.0,163376.0,304833.0,775658.0,9213680.0,452344.0,10910586.0,3.7,2.0,3.3,3.0,2.6,63.0,0.019912,1.0,1007.0,2.0,1.0,1.0,0.0,0.0,2020,0.000000,1.978198e+11,7303.640282
12629,ID_zyfj4W,1,8811.121632,8832.560173,8754.602574,2.412260e+09,1.621298e+11,1653.0,757.0,74.0,249.0,60.0,107.0,4852.0,513.0,14878543.0,92.0,1127.0,454.0,3252.0,144.0,203.0,952.0,3101.0,452.0,617875.0,656816.0,2617007.0,10307241.0,684529.0,14885477.0,3.7,44.0,2.8,3.0,3.9,67.0,0.020715,1.0,918.0,2.0,1.0,1.0,1.0,2.0,5787,0.000000,2.493421e+11,8828.440234
12630,ID_zz5eGB,1,9661.514681,9662.196605,9645.133934,9.861517e+08,1.774933e+11,1376.0,657.0,39.0,81.0,43.0,122.0,3915.0,675.0,14439095.0,153.0,871.0,274.0,3343.0,74.0,188.0,671.0,2691.0,291.0,278732.0,360131.0,1733775.0,10881941.0,1189157.0,14445315.0,3.8,25.0,3.0,4.4,4.5,78.5,0.012411,1.0,603.0,2.0,1.0,1.0,0.0,9.0,4679,2.706432,2.757547e+11,9658.204385


In [171]:
test

,id,asset_id,open,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_volume,percent_change_24h,market_cap_global
0,ID_323Sn2,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,7.0,56.0,2.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0.000000,0.000000e+00
1,ID_325SNW,1,11335.062188,11351.690956,11335.062188,1.064152e+09,2.101463e+11,1664.0,1045.0,64.0,213.0,51.0,274.0,6046.0,3034.0,23453171.0,119.0,2305.0,1367.0,6252.0,151.0,565.0,603.0,4553.0,174.0,2900568.0,1898920.0,2268741.0,14056214.0,2338771.0,23465365.0,3.7,39.0,3.2,3.4,2.8,65.5,0.004407,1.0,711.0,2.0,1.0,1.0,1.0,1.0,7245,-0.555698,3.631052e+11
2,ID_325uzE,1,6322.560756,6328.362354,6294.714484,1.516268e+09,1.153862e+11,397.0,255.0,11.0,72.0,30.0,112.0,2404.0,304.0,3831278.0,12.0,346.0,73.0,604.0,39.0,103.0,668.0,1406.0,188.0,29147.0,411178.0,873284.0,2389256.0,129448.0,3832828.0,3.7,2.0,3.0,3.0,3.4,65.5,0.024035,1.0,715.0,2.0,1.0,1.0,0.0,0.0,2702,1.689370,1.771075e+11
3,ID_328qCx,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,8.0,96.0,217.0,244.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225,0.000000,0.000000e+00
4,ID_3293uJ,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,26.0,49.0,33.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,ID_zufSPk,1,8266.942912,8266.942912,8259.796103,3.100637e+08,1.491229e+11,618.0,438.0,15.0,202.0,42.0,164.0,996.0,262.0,3310472.0,19.0,217.0,135.0,715.0,2.0,25.0,273.0,671.0,25.0,987.0,187128.0,1174018.0,1933268.0,16157.0,3318645.0,3.7,3.0,2.4,2.3,3.9,61.5,0.007076,1.0,921.0,2.0,1.0,1.0,0.0,0.0,1494,-0.967973,2.262125e+11
6218,ID_zuz9yf,1,10869.612767,10873.493490,10865.827625,6.989360e+08,2.011601e+11,1608.0,911.0,84.0,204.0,45.0,123.0,5994.0,3808.0,30985192.0,219.0,3020.0,1416.0,7673.0,196.0,502.0,605.0,4484.0,207.0,14614866.0,1739950.0,2052440.0,12246575.0,343689.0,30999455.0,3.7,31.0,2.9,3.9,3.0,67.5,0.005486,1.0,807.0,2.0,1.0,1.0,0.0,6.0,7065,0.000000,3.523272e+11
6219,ID_zvrMSX,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,8.0,139.0,167.0,317.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,175,0.000000,0.000000e+00
6220,ID_zy9Cfv,1,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,6.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,0.000000,0.000000e+00


check duplicate

In [172]:
# Check for duplicates
train.duplicated().any(), test.duplicated().any()

(False, False)

In [173]:
# Select main columns to be used in training
features=['open','high','low','social_score','average_sentiment','volume','news','price_score','social_impact_score','correlation_rank','galaxy_score','volatility','market_cap_rank','percent_change_24h_rank','social_volume','market_cap','percent_change_24h','market_cap_global']
X = train[features]
y = train.close.astype(float)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=1994)

# machine learning

In [174]:
X

,open,high,low,social_score,average_sentiment,volume,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,market_cap_rank,percent_change_24h_rank,social_volume,market_cap,percent_change_24h,market_cap_global
0,9422.849081,9428.490628,9422.849081,11681999.0,3.6,7.131986e+08,69.0,2.7,3.6,3.3,66.0,0.007118,1.0,606.0,4422,1.737635e+11,1.434516,2.818066e+11
1,7985.359278,7992.059917,7967.567267,5924770.0,3.7,4.004755e+08,1.0,2.0,2.0,1.0,43.5,0.009419,1.0,0.0,2159,1.426942e+11,-2.459507,2.126897e+11
2,49202.033778,49394.593518,49068.057046,47796942.0,3.7,3.017729e+09,22.0,3.1,3.0,3.3,65.5,0.013530,1.0,692.0,10602,9.166977e+11,4.942448,1.530712e+12
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,285,0.000000e+00,0.000000,0.000000e+00
4,10535.737119,10535.737119,10384.798216,14426405.0,3.7,1.150053e+09,22.0,4.7,3.8,4.4,83.0,0.010332,1.0,749.0,3996,1.921183e+11,2.609576,3.386925e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12627,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,140,0.000000e+00,0.000000,0.000000e+00
12628,7294.506136,7306.701928,7292.729876,10910586.0,3.7,3.816704e+08,2.0,3.3,3.0,2.6,63.0,0.019912,1.0,1007.0,2020,1.322473e+11,0.000000,1.978198e+11
12629,8811.121632,8832.560173,8754.602574,14885477.0,3.7,2.412260e+09,44.0,2.8,3.0,3.9,67.0,0.020715,1.0,918.0,5787,1.621298e+11,0.000000,2.493421e+11
12630,9661.514681,9662.196605,9645.133934,14445315.0,3.8,9.861517e+08,25.0,3.0,4.4,4.5,78.5,0.012411,1.0,603.0,4679,1.774933e+11,2.706432,2.757547e+11


In [175]:
from sklearn.ensemble import StackingRegressor

In [176]:
ss =pd.read_csv("/content/drive/MyDrive/ZINDI/IEEE Hackathon/SampleSubmission (2).csv")

In [177]:
nfolds = 10
kfolds=KFold(n_splits=nfolds, shuffle=True, random_state=1994)

In [178]:
models = [('xgb',XGBRegressor(learning_rate=0.01,n_estimators=20000,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.006)),
          ('linearregression',LinearRegression())]

In [ ]:
y_pred_tot=[]
for ind,(ind_train,ind_val) in (enumerate (kfolds.split(X,y))): 
    X_train,X_val = X.iloc[ind_train],X.iloc[ind_val] 
    y_train,y_val = y.iloc[ind_train],y.iloc[ind_val]
    model = StackingRegressor(estimators=models)
    model.fit(X_train,y_train)
    val_pred = model.predict(X_val)
    score_val=np.sqrt(mean_squared_error(y_val, val_pred))
    print(score_val)
    test_pred = model.predict(test[features])
    y_pred_tot.append(test_pred)
    

In [149]:
len(y_pred_tot)

10

In [150]:
y_pred_tot

[array([3.67498374e-01, 1.13508389e+04, 6.31039975e+03, ...,
        4.29263422e-01, 3.66382138e-01, 7.06237159e+03]),
 array([-1.34169311e+00,  1.13513677e+04,  6.30601609e+03, ...,
        -1.34141040e+00, -1.34169822e+00,  7.06715736e+03]),
 array([-2.52311854e-01,  1.13519763e+04,  6.30528140e+03, ...,
        -2.51724840e-01, -2.52320617e-01,  7.06741246e+03]),
 array([1.00350990e+00, 1.13514636e+04, 6.30403556e+03, ...,
        1.02902383e+00, 1.00304881e+00, 7.06325878e+03]),
 array([1.96933672e+00, 1.13513779e+04, 6.31871705e+03, ...,
        1.95523703e+00, 1.96959153e+00, 7.06954231e+03]),
 array([-1.79136883e+00,  1.13489433e+04,  6.31063018e+03, ...,
        -1.78676787e+00, -1.79145198e+00,  7.07172292e+03]),
 array([1.45160050e+00, 1.13485425e+04, 6.30254800e+03, ...,
        1.46914317e+00, 1.45128346e+00, 7.06245564e+03]),
 array([3.22999668e+00, 1.13508461e+04, 6.30763113e+03, ...,
        3.26567856e+00, 3.22935183e+00, 7.06443557e+03]),
 array([-7.58958364e-01,  1.13

In [156]:
predictions=np.mean(y_pred_tot,0)
sub_file = ss.copy()
sub_file.close = predictions
sub_file.to_csv('submit.csv', index = False)